In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import misc
import skimage as sk
from skimage import io
from skimage import transform

from sklearn import metrics as ms
from sklearn import neural_network
from sklearn import ensemble
from sklearn import tree
from sklearn import svm


from sklearn.cross_validation import train_test_split
from PIL import Image
from matplotlib.pyplot import imshow
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D
from keras.utils import np_utils

/home/david/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
ls

beHappy.csv                testData/   train_labels.csv  usedTrainData.csv
Happyness detection.ipynb  trainData/  usedTestData.csv


In [3]:
df = pd.read_csv('train_labels.csv') # contain train data ID and label
df.head()

,ID,isHappy
0,1,0
1,2,0
2,3,1
3,4,1
4,5,1


In [4]:
image = misc.imread('trainData/1.png')
imgArray = sk.color.rgb2grey(image) # convert to gray
imgArray =  sk.transform.resize(imgArray, (12,12))
imgArray

/home/david/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


array([[ 0.0872549 ,  0.05980392,  0.08333333,  0.05490196,  0.26372549,
         0.37156863,  0.14215686,  0.33431373,  0.32254902,  0.87941176,
         0.88137255,  0.82058824],
       [ 0.11078431,  0.08627451,  0.04411765,  0.05784314,  0.03921569,
         0.0745098 ,  0.06764706,  0.13333333,  0.27156863,  0.84803922,
         0.88137255,  0.82647059],
       [ 0.11372549,  0.1627451 ,  0.13039216,  0.13529412,  0.08333333,
         0.09019608,  0.07745098,  0.21568627,  0.30098039,  0.77941176,
         0.88333333,  0.83333333],
       [ 0.55294118,  0.10196078,  0.05490196,  0.06568627,  0.72941176,
         0.72745098,  0.30196078,  0.61666667,  0.88333333,  0.8754902 ,
         0.87352941,  0.84019608],
       [ 0.54117647,  0.71764706,  0.26666667,  0.76764706,  0.76470588,
         0.40784314,  0.50392157,  0.69705882,  0.73921569,  0.8745098 ,
         0.87058824,  0.83039216],
       [ 0.31666667,  0.78529412,  0.78627451,  0.73431373,  0.77058824,
         0.5627451 ,  

In [5]:
imageG = sk.io.imread('trainData/1.png')
imageG = sk.color.rgb2grey(imageG)

#misc.imshow(imageG)

In [7]:
def getImageAsVector(image):
    imgArray = sk.io.imread(image) # load as matrix
    imgArray = sk.color.rgb2grey(imgArray) # convert to gray
    
    imgShape = imgArray.shape
    imgArray = imgArray.reshape(imgShape[0]*imgShape[1]) # convert to vector format
    
    return list(imgArray)


def matrixToDataframe(data, colNames = None):
    if colNames is None:
        colNames = np.arange(data.shape[1]).astype(str)

    df = pd.DataFrame( data, columns = colNames)

    return df


def images2Dataframe(labelFiel, folder):
    """
    Convert imgs in a folder to vectors. Add label to images according to the label file.
    Return as a dataframe contain the vectors, and labels.
    """
    # Read the csv conatin img IDs and labes.
    df = pd.read_csv(labelFiel)
    cols = df.columns
    ID = cols[0]
    target = cols[1]
    imgList = list(df[ID])
    
    # Craet MAtrix with zeroes will conatin all images as a vecor
    numOfImgs = len(imgList) # get the num of images
    tmpImg = getImageAsVector(folder + str(imgList[1]) + '.png')
    size = len(tmpImg) # Get the img vecot size
    M = np.zeros([numOfImgs, size]) # Matrix contain all image

    # Add values vector values to matrix
    for i in range(numOfImgs):
        imgName = folder + str(imgList[i]) + '.png'
        M[i,:] = getImageAsVector(imgName)
        
    imgDf = matrixToDataframe(M) # convert matrix to dataframe
    features = imgDf.columns
    imgDf[cols] = df[cols] # add label and ID to image dataframe   
    
    return imgDf, features, target, ID

In [8]:

imgDf, features, target, ID = images2Dataframe('train_labels.csv', 'trainData/')
#imgDf.to_csv('usedTrainData.csv', index=False)

#imdDf = pd.read_csv('usedTrainData.csv')
imgDf.head()

,0,1,2,3,4,5,6,7,8,9,...,2296,2297,2298,2299,2300,2301,2302,2303,ID,isHappy
0,0.058824,0.098039,0.121569,0.090196,0.098039,0.050980,0.035294,0.039216,0.090196,0.149020,...,0.631373,0.631373,0.635294,0.654902,0.670588,0.690196,0.694118,0.690196,1,0
1,0.090196,0.074510,0.086275,0.094118,0.113725,0.176471,0.203922,0.203922,0.227451,0.270588,...,0.341176,0.439216,0.415686,0.447059,0.462745,0.470588,0.505882,0.552941,2,0
2,0.400000,0.407843,0.411765,0.411765,0.407843,0.392157,0.325490,0.298039,0.250980,0.254902,...,0.701961,0.756863,0.858824,0.862745,0.847059,0.627451,0.356863,0.360784,3,1
3,0.384314,0.380392,0.407843,0.541176,0.450980,0.454902,0.600000,0.698039,0.698039,0.768627,...,0.498039,0.478431,0.454902,0.443137,0.400000,0.250980,0.333333,0.380392,4,1
4,0.015686,0.007843,0.066667,0.231373,0.325490,0.352941,0.376471,0.478431,0.513725,0.352941,...,0.439216,0.411765,0.388235,0.537255,0.145098,0.011765,0.035294,0.062745,5,1


In [9]:

testDf, features, target, ID = images2Dataframe('beHappy.csv', 'testData/')
#imgDf.to_csv('usedTestData.csv', index=False)

#testDf = pd.read_csv('usedTestData.csv')
testDf.head()

,0,1,2,3,4,5,6,7,8,9,...,2296,2297,2298,2299,2300,2301,2302,2303,ID,isHappy
0,0.552941,0.611765,0.658824,0.658824,0.694118,0.780392,0.866667,0.713725,0.498039,0.392157,...,0.592157,0.011765,0.192157,0.231373,0.231373,0.219608,0.298039,0.309804,10001,1.0
1,0.839216,0.843137,0.847059,0.854902,0.854902,0.854902,0.854902,0.866667,0.792157,0.349020,...,0.200000,0.239216,0.239216,0.286275,0.388235,0.823529,0.949020,0.952941,10002,0.0
2,0.019608,0.509804,0.674510,0.674510,0.752941,0.772549,0.772549,0.780392,0.815686,0.780392,...,0.003922,0.003922,0.003922,0.003922,0.011765,0.019608,0.047059,0.101961,10003,0.0
3,0.050980,0.043137,0.066667,0.090196,0.125490,0.172549,0.125490,0.074510,0.078431,0.125490,...,0.294118,0.278431,0.282353,0.298039,0.309804,0.313725,0.184314,0.117647,10004,1.0
4,0.729412,0.643137,0.729412,0.772549,0.823529,0.631373,0.572549,0.678431,0.650980,0.411765,...,0.478431,0.325490,0.239216,0.380392,0.443137,0.501961,0.388235,0.294118,10005,0.0


In [11]:
def splitTrainTest(df, rate, minLength):
    randomIndex = np.arange(minLength)
    np.random.shuffle(randomIndex)
    randomIndex = randomIndex.tolist()
    splitPlace = int(np.floor(len(df)*rate)) # min 1 test beacuse of floor
    train = df.iloc[randomIndex[:splitPlace]]
    test = df.iloc[randomIndex[splitPlace:]]

    return train, test

def splitTrainTestEqual(df, target, rate):
    labels = list(set(df[target]))

    testList = []
    trainList = []

    minLength = 1e28
    for l in labels:
        tmpDf = df[df[target] == l]
        if minLength > len(tmpDf):
            minLength = len(tmpDf)

    for l in labels:
        tmpDf = df[df[target] == l]
        tmpTrain, tmpTest = splitTrainTest(tmpDf, rate, minLength)
        testList.append(tmpTest)
        trainList.append(tmpTrain)

    test = pd.concat(testList, ignore_index = True)
    train = pd.concat(trainList, ignore_index = True)

    test = pd.DataFrame(test.values, columns = df.columns)
    train = pd.DataFrame(train.values, columns = df.columns)

    return train, test

def classification(df, features, target, clf, iteration = 1, rate = 0.8):
    precision = 0

    for i in range(iteration):
        train, test = splitTrainTestEqual(df, target, rate)

        clf.fit( train[features].values , train[target].values)

        predictions = clf.predict(test[features].values)
        precision += ms.accuracy_score(test[target].values,predictions)

    precision /= iteration

    return precision, clf

In [12]:
#clf = neural_network.MLPClassifier(solver='lbfgs', hidden_layer_sizes=(20,), max_iter = 500)
clf = svm.SVC(gamma=0.01)
precision, clf = classification(imgDf, features, target, clf)
precision

KeyboardInterrupt: 

In [75]:
predictions = clf.predict(testDf[features].values)
testDf[target] = predictions

output = testDf[[ID, target]]
output.to_csv('beHappy.csv', index = False)
output.head()

,ID,isHappy
0,10001,1.0
1,10002,0.0
2,10003,0.0
3,10004,1.0
4,10005,0.0


In [27]:
data  = imgDf[list(features) + [target]].values
data.shape

(10000, 2305)